# Pipeline and conversation experiments

## Google translate

In [1]:
from googletrans import Translator

translator = Translator()
def sv_to_en(text):
    # Translates text from swedish to english
    out = translator.translate(text,src='sv',dest='en')
    return out.text

def en_to_sv(text):
    # Translate text from english to swedish
    out = translator.translate(text,src='en',dest='sv')
    return out.text


# Blenderbot

In [2]:
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration
from torch import no_grad
mname = 'facebook/blenderbot-1B-distill' # options: 'facebook/blenderbot_small-90M' , 'facebook/blenderbot-400M-distill' ,'facebook/blenderbot-3B'
model = BlenderbotForConditionalGeneration.from_pretrained(mname)
tokenizer = BlenderbotTokenizer.from_pretrained(mname)



In [3]:

## TODO for easier testing
class BlenderConversation:
    
    def __init__(self,user_input=None):
        self.len = 0
        self.user_inputs = []
        if user_input not None:
            self.user_inputs.append(user_inputs)
            self.user_turn = False
        else:
            self.user_turn = True
        self.bot_replies = []
        
        
    def add_user_input(self,text):
        if self.user_turn:
            self.user_inputs.append(text)
        else:
            raise ValueError("It's the bot's turn to add a reply to the conversation")

    
    def add_bot_reply(self,text):
        if not self.user_turn:
            self.bot_replies.append(text)
        else:
            raise ValueError("It's the user's turn to add an input to the conversation
        
    def get_dialogue_history(self,max_len=128):
        # Returns string of the dialogue history with bot and user inputs separated with '\n'
        # Todo
        history = ''
        tokens_left = max_len
        if self.user_turn:
            
        else:
        
        
    def to_txt(self):
    



def strip_token(line):
    line = line.replace('<s>','')
    line = line.replace('</s>','')
    return line

def save_blender_convo(convo_list):
    s = ''
    for line in convo_list:
        s = s + '>>> ' + line + '\n'
    s = s + '\n'
    with open('01_conversation_output/blender_convo.txt','a') as f:
        f.write(s)
        

## Dialogue history chatting helper

In [4]:

def blender_history_chatting(user_input):
    UTTERANCE = sv_to_en(user_input)
    
    convo_list_swe.append(user_input)
    convo_list.append(UTTERANCE)
    
    # Check that conversation isn't longer than max_len=128 words 
    utter_len = len(UTTERANCE.split())
    convo_len = 0
    for words in convo_list:
        convo_len += len(words.split())
    
    if convo_len > 128 - utter_len:
        while convo_len > 128:
            throw_away= convo_list.pop(0)
            convo_len -= len(throw_away.split())

    context = ''
    for i,line in enumerate(convo_list):
        if i == len(convo_list) -1:
            context = context + line
        else:
            context = context + line + '\n'

    with no_grad():
        inputs = tokenizer([context], return_tensors='pt')
        reply_ids = model.generate(**inputs)

        reply = strip_token(tokenizer.batch_decode(reply_ids)[0])
    convo_list.append(reply)

    convo_list_swe.append(en_to_sv(reply))
    for line in convo_list_swe:
        print(line,'\n')
   


## 1. Dialogue history chatting - model given dialogue as context

In [18]:
# Reset convo lists
convo_list = []
convo_list_swe = []


In [21]:
user_input = 'Jag ska ut och springa '

blender_history_chatting(user_input)

Hej Melina! 

Hej! Hur mår du idag? Jag mår bra, tack. Och du då? 

Jag mår bra! 

Det är fantastiskt att höra! Jag mår bra också. Vad ska du göra idag? 

Jag ska ut och springa  

Toppen! Jag kom precis tillbaka från en körning själv. Hur länge har du kört? 



## Save convo to text file

In [ ]:
save_blender_convo(convo_list_swe)

## Context chatting helper

In [6]:
def blender_context_chatting(user_input):
    # Translation and conversation recording
    UTTERANCE = sv_to_en(user_input)
    convo_list_swe.append(user_input)

    model_input = context + '\n' + UTTERANCE

    # To model
    with no_grad():
        inputs = tokenizer([model_input], return_tensors='pt')
        reply_ids = model.generate(**inputs)
        reply = strip_token(tokenizer.batch_decode(reply_ids)[0])

    # Add reply to conversation and print conversation history
    convo_list.append(reply)
    convo_list_swe.append(en_to_sv(reply))
    for i, line in enumerate(convo_list_swe):
        if i % 2:
            print('bot >>> ',line,'\n')
        else:
            print('user >>> ',line,'\n')



## 2. Context chatting - bot has access to the context text and the most recent user input

In [11]:
context = 'Jag är här på intervju. Jag söker ett jobb som städare. Jag har erfarenhet från två tidigare jobb.'
context = sv_to_en(context)

# Reset conversation lists
convo_list_swe = []

In [17]:
user_input= 'Tack, ger du mig det?'
blender_context_chatting(user_input)

user >>>  Hej  

bot >>>  Hej hur mår du? Jag letar efter ett jobb som städare. Jag har erfarenhet från två tidigare jobb. 

user >>>  Nej jag söker jobb. DU ska intervjua mig 

bot >>>  Jag är här för en intervju. Jag letar efter ett jobb som städare. Jag har erfarenhet från två tidigare jobb. 

user >>>  Jag är från KIna 

bot >>>  Det är coolt. Jag har aldrig varit i Kina. Vad arbetar du med? 

user >>>  Jag är arbetssökande 

bot >>>  Toppen! Jag hoppas att du får jobbet. Vilken typ av jobb letar du efter? 

user >>>  Tack, ger du mig det? 

bot >>>  Nej, jag ger det inte till dig. Jag letar efter ett jobb som städare. Jag har erfarenhet från två tidigare jobb. 



## Save conversation to text file 

In [62]:
save_blender_convo(convo_list_swe)